In [1]:
import gc
import numpy as np
import pandas as pd
import lightgbm as lgb
from time import time
from datetime import date
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.set_option('display.max_columns', 1000)

# 1 Preprocessing

In [8]:
inv = pd.read_csv(
    "../../data/level2/m111-sku-inv.csv", sep=',', 
    parse_dates=['period_wid'], dtype={'item_code': str}
).rename(columns={'period_wid': 'inv_date'})

In [9]:
inv = inv.loc[inv.inv_date.isin(pd.date_range('2017-06-30', '2018-12-31', freq='M'))]

In [10]:
inv['inv_month'] = inv.inv_date.astype('str').apply(lambda x: x[:7])

In [11]:
len(inv)

62397

In [12]:
category = pd.read_csv(
    "../../data/level2/item2category-from-3.csv", sep=',', dtype={'item_code': str}
).rename(columns={'sales_segment1_code': 'category'})

In [13]:
len(category)

5079

In [18]:
inv = inv.join(category.set_index('item_code'), on='item_code', how='left')

In [19]:
len(inv)

62397

In [24]:
inv.category.fillna(-1, inplace=True)

In [28]:
inv_month = inv.groupby(['category', 'inv_month'])[['qty']].sum()

In [29]:
inv_month.head()

qty
category inv_month        
-1       2017-12    6821.0
         2018-01      82.0
         2018-02     106.0
         2018-03     194.0
         2018-04     179.0

In [30]:
inv_month_hori = inv_month.unstack(level=-1).fillna(0)

In [32]:
inv_month_hori.columns = inv_month_hori.columns.get_level_values(1)

In [34]:
inv_month_hori.to_csv("inv.csv")